In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('../data/dataset/QTS_data_001.csv', index_col = 0)

In [6]:
def cat_col(data):
    data['요일'] = data['요일'].astype('category')
    data['채널명'] = data['채널명'].astype('category')

    return data

In [7]:
df = cat_col(df)

In [8]:
X = df.drop(['label'],axis=1)
y = df['label']

X_train, X_valid, y_train, y_valid = train_test_split(X,y,  test_size=0.2,  shuffle=True, random_state=1004)

In [9]:
from lightgbm import LGBMClassifier
import optuna
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


def objectives(trial):
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 300, 8000),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
            'objective' : 'binary',
    }

    model = LGBMClassifier(**params)
    model.fit(X_train,y_train)
    y_valid_pred = cross_val_predict(model, X_valid, y_valid, cv=3)
    ac = accuracy_score(y_valid, y_valid_pred)
    print('accuracy : ', ac)
    return ac

In [10]:
studyLGBM = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
studyLGBM.optimize(objectives, n_trials=20)

trial = studyLGBM.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0

model_o = LGBMClassifier(**params_best)#

[I 2022-05-11 16:14:21,982] A new study created in memory with name: no-name-c95bb047-757b-489b-b3b9-1775f6aac2d7


KeyboardInterrupt: 

In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

In [ ]:
#파라미터 중요도 시각화
optuna.visualization.plot_param_importances(studyLGBM)